In [25]:
#import some necessary lib
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
#the src of the image
img_src = 'D:/VOC_HY/image_test/'
num_part = 3
os.chdir(img_src)
im1 = Image.open(img_src + 's62_000000.jpg')
im2 = Image.open(img_src + 's63_000025.jpg')

#compute the coordiante of the sub_image
crop_dic= []#put the coordiantes in the list
width = im1.size[0]/num_part
height = im1.size[1]/num_part
for i in range(num_part):
    for j in range(num_part):
        crop_dic.append((i*width,j*height,i*width+width,j*height+height))
i=0
print len(crop_dic)
for t in crop_dic:
    im_new = im1.crop(t)
    im_new.save('./new_file/im1_'+str(i)+'.jpg')
    i = i + 1
i=0
for t in crop_dic:
    im_new = im2.crop(t)
    im_new.save('./new_file/im2_'+str(i)+'.jpg')
    i = i + 1


9


In [26]:
split_image_src = 'D:/VOC_HY/image_test/new_file'
image_list = os.listdir(split_image_src)
list_afterSort = []
for i in range(9):
    list_afterSort.append(image_list[i])
    list_afterSort.append(image_list[i+9])
print list_afterSort

['im1_0.jpg', 'im2_0.jpg', 'im1_1.jpg', 'im2_1.jpg', 'im1_2.jpg', 'im2_2.jpg', 'im1_3.jpg', 'im2_3.jpg', 'im1_4.jpg', 'im2_4.jpg', 'im1_5.jpg', 'im2_5.jpg', 'im1_6.jpg', 'im2_6.jpg', 'im1_7.jpg', 'im2_7.jpg', 'im1_8.jpg', 'im2_8.jpg']


In [45]:
# -*- coding: utf-8 -*-
import caffe
import numpy as np
import pylab


root='D:\VOC_HY/cjDATA/new_data/'   #根目录
deploy=root + 'squeezeNetFeatureExtract/deploy.prototxt'    #deploy文件
caffe_model=root + 'squeezeNetFeatureExtract/squeezenet_v1.0.caffemodel'   #训练好的 caffemodel

net = caffe.Net(deploy,caffe_model,caffe.TEST)   #加载model和network


#图片预处理设置
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})  #设定图片的shape格式(1,3,28,28)
transformer.set_transpose('data', (2,0,1))    #改变维度的顺序，由原始图片(28,28,3)变为(3,28,28)
#transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))    #减去均值，前面训练模型时没有减均值，这儿就不用
transformer.set_raw_scale('data', 1)    # 缩放到【0，255】之间
transformer.set_channel_swap('data', (2,1,0))   #交换通道，将图片由RGB变为BGR

img_src ='D:/VOC_HY/image_test/'
new = np.zeros((1,1000))
list_new = os.listdir('D:/VOC_HY/image_test')
for img_file in list_new:
    img = img_src + img_file
    print img
    im = caffe.io.load_image(img)  # 加载图片
    im = im/255
    net.blobs['data'].data[...] = transformer.preprocess('data', im)  # 执行上面设置的图片预处理操作，并将图片载入到blob中
    out = net.forward()
    feat = np.array(out['prob']).reshape((1, 1000))
    new = np.vstack((new, feat))

new = new[1:,:]
totalNum = 1
for i in range(totalNum):
    x = 100* np.sqrt(np.sum(np.square(new[i] - new[i+1])))
    print ((i),(i+1)),'  ',x,
    if(x>0.01):
        x=False
    else:
        x = True
    print x

D:/VOC_HY/image_test/s62_000012.jpg
D:/VOC_HY/image_test/s63_000047.jpg
(0, 1)    0.012575957585 False


In [39]:
# -*- coding: utf-8 -*-
import caffe
import numpy as np
import pylab


root='D:/VOC_HY/cjDATA/new_data/'   #根目录
deploy=root + 'squeezeNetFeatureExtract/deploy.prototxt'    #deploy文件
caffe_model=root + 'squeezeNetFeatureExtract/squeezenet_v1.0.caffemodel'   #训练好的 caffemodel

net = caffe.Net(deploy,caffe_model,caffe.TEST)   #加载model和network


#图片预处理设置
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})  #设定图片的shape格式(1,3,28,28)
transformer.set_transpose('data', (2,0,1))    #改变维度的顺序，由原始图片(28,28,3)变为(3,28,28)
#transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))    #减去均值，前面训练模型时没有减均值，这儿就不用
transformer.set_raw_scale('data', 1)    # 缩放到【0，255】之间
transformer.set_channel_swap('data', (2,1,0))   #交换通道，将图片由RGB变为BGR

img_src ='D:/VOC_HY/image_test/'
new = np.zeros((1,1000))
list_afterSort = os.listdir(img_src)
for idx, imgFile in enumerate(list_afterSort):
    img = img_src + imgFile
    print img
    im = caffe.io.load_image(img)  # 加载图片
    im = im /255
    net.blobs['data'].data[...][idx] = transformer.preprocess('data', im)  # 执行上面设置的图片预处理操作，并将图片载入到blob中
out = net.forward()
np.sum(np.square(out['prob'][1]-out['prob'][0]))

D:/VOC_HY/image_test/s63_000025.jpg
D:/VOC_HY/image_test/s63_000047.jpg


1.5855456e-09